# Homework Week 4

## Question 1

In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from rouge import Rouge

/usr/local/python/3.10.13/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)
df = df.iloc[:300]

In [3]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [4]:
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

In [5]:
answer_llm = df.iloc[0].answer_llm
embedding = embedding_model.encode(answer_llm)

In [6]:
embedding[0]

np.float32(-0.42244655)

**Answer = -0.42**

## Question 2

In [7]:
for i, row in tqdm(df.iterrows()):
    embedding_answer_llm = embedding_model.encode(row.answer_llm)
    embedding_answer_orig = embedding_model.encode(row.answer_orig)  
    df.at[i, "cosine"] = embedding_answer_llm.dot(embedding_answer_orig)

0it [00:00, ?it/s]

In [8]:
df["cosine"].describe()

count    300.000000
mean      27.495996
std        6.384742
min        4.547924
25%       24.307844
50%       28.336870
75%       31.674309
max       39.476013
Name: cosine, dtype: float64

In [9]:
np.percentile(df["cosine"], 75)

np.float32(31.674309)

**Answer = 31.67**

## Question 3

In [10]:
def vector_normalised(vector):
    norm = np.sqrt((vector * vector).sum())
    vector_norm = vector / norm
    
    return vector_norm  

In [11]:
for i, row in tqdm(df.iterrows()):
    embedding_answer_llm = embedding_model.encode(row.answer_llm)
    embedding_answer_orig = embedding_model.encode(row.answer_orig)

    embedding_answer_llm_normalised = vector_normalised(embedding_answer_llm)
    embedding_answer_orig_normalised = vector_normalised(embedding_answer_orig)
    
    df.at[i, "cosine_normalised"] = embedding_answer_llm_normalised.dot(embedding_answer_orig_normalised)

0it [00:00, ?it/s]

In [12]:
df["cosine_normalised"].describe()

count    300.000000
mean       0.728393
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: cosine_normalised, dtype: float64

In [13]:
np.percentile(df["cosine_normalised"], 75)

np.float32(0.8362349)

**Answer = 0.83**

## Question 4

In [14]:
rouge_test = df.iloc[10]
rouge_test

answer_llm           Yes, all sessions are recorded, so if you miss...
answer_orig          Everything is recorded, so you won’t miss anyt...
document                                                      5170565b
question                          Are sessions recorded if I miss one?
course                                       machine-learning-zoomcamp
cosine                                                       32.344711
cosine_normalised                                             0.777956
Name: 10, dtype: object

In [15]:
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(rouge_test['answer_llm'], rouge_test['answer_orig'])[0]

In [16]:
scores["rouge-1"]['f']

0.45454544954545456

**Answer = 0.45**

## Question 5

In [17]:
scores_avg = [scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f']]
avg = np.mean(scores_avg)

In [18]:
avg

np.float64(0.35490034990035496)

**Answer = 0.35**

## Question 6

In [19]:
f_scores = []

for row in df.iterrows():
    scores = rouge_scorer.get_scores(row[1]['answer_llm'], row[1]['answer_orig'])[0]
    f_scores.append({k: scores[k]['f'] for k in ['rouge-1', 'rouge-2', 'rouge-l']})

df_scores = pd.DataFrame(f_scores)
df_scores['rouge_avg'] = df_scores.mean(axis=1)
df_scores

,rouge-1,rouge-2,rouge-l,rouge_avg
0,0.095238,0.028169,0.095238,0.072882
1,0.125000,0.055556,0.093750,0.091435
2,0.415584,0.177778,0.389610,0.327658
3,0.216216,0.047059,0.189189,0.150821
4,0.142076,0.033898,0.120219,0.098731
...,...,...,...,...
295,0.654545,0.540984,0.618182,0.604570
296,0.590164,0.460432,0.557377,0.535991
297,0.654867,0.564516,0.637168,0.618851
298,0.304762,0.132231,0.304762,0.247252


In [20]:
df_scores['rouge-2'].mean()

np.float64(0.20696501983423318)

**Answer = 0.20**